In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Download stock data from Yahoo Finance
def download_stock_data(ticker, period='5y'):
    stock_data = yf.download(ticker, period=period)
    return stock_data


import requests


# Step 2: Preprocess the data
def preprocess_data(data, feature_col='Close', seq_length=60):
    # Use 'Close' prices to predict trends
    data = data[[feature_col]]

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences of data points for LSTM input
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)

    # Reshape the data to be compatible with LSTM (samples, timesteps, features)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler

# Step 3: Build the LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))  # Predicting a single output value (next price)

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the LSTM model
def train_lstm_model(model, X_train, y_train, epochs=50, batch_size=64):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

# Step 5: Make predictions and evaluate
def predict_and_evaluate(model, X_test, y_test, stock_scaler):
    predictions = model.predict(X_test)

    # Inverse transform only the stock price column
    predictions_stock_price = stock_scaler.inverse_transform(predictions)
    y_test_stock_price = stock_scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate the Root Mean Squared Error (RMSE) on the stock prices
    rmse = np.sqrt(np.mean((predictions_stock_price - y_test_stock_price) ** 2))
    return predictions_stock_price, rmse

# Step 6: Trading simulation logic
def simulate_trading(predictions, actual_prices, dates, initial_balance=10000, shares=0):
    balance = initial_balance
    total_shares = shares
    trade_log = []

    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        actual_price = actual_prices[i]
        date = dates[i]  # Make sure 'date' is a datetime object

        # Ensure you're not comparing dates with prices
        if predicted_price > actual_prices[i-1] and balance > actual_price:
            shares_to_buy = balance // actual_price
            balance -= shares_to_buy * actual_price
            total_shares += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_prices[i][0]:.2f} on {dates[i]}")

        elif predicted_price < actual_prices[i-1] and total_shares > 0:
            balance += total_shares * actual_price
            trade_log.append(f"Sold {total_shares} shares at {actual_price[0]:.2f} on {date}, Balance: {balance:.2f}")
            total_shares = 0

    # Final balance after selling any remaining shares
    if total_shares > 0:
        balance += total_shares * actual_prices[-1]
        trade_log.append(f"Final Sale of {total_shares} shares at {actual_prices[-1][0]:.2f} on {dates[-1]}, Final Balance: {balance:.2f}")


    profit_loss = balance - initial_balance
    return trade_log, profit_loss


In [ ]:
# Step 6: Trading simulation logic
def simulate_trading(predictions, actual_prices, dates, initial_balance=10000, shares=0):
    balance = initial_balance
    total_shares = shares
    trade_log = []

    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        actual_price = actual_prices[i]
        date = dates[i]  # Make sure 'date' is a datetime object

        # Ensure you're not comparing dates with prices
        if predicted_price > actual_prices[i-1] and balance > actual_price:
            shares_to_buy = balance // actual_price
            balance -= shares_to_buy * actual_price
            total_shares += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_price} on {date}, Balance: {balance}, Shares: {total_shares}")

        elif predicted_price < actual_prices[i-1] and total_shares > 0:
            balance += total_shares * actual_price
            trade_log.append(f"Sold {total_shares} shares at {actual_price} on {date}, Balance: {balance}")
            total_shares = 0

    # Final balance after selling any remaining shares
    if total_shares > 0:
        balance += total_shares * actual_prices[-1]
        trade_log.append(f"Final Sale of {total_shares} shares at {actual_prices[-1]} on {dates[-1]}, Final Balance: {balance}")

    profit_loss = balance - initial_balance
    return trade_log, profit_loss

# Step 7: Run the entire pipeline with trading simulation
def run_stock_prediction_with_simulation(ticker, period='5y', seq_length=60):
    # Download and preprocess the data
    stock_data = download_stock_data(ticker, period)
    X, y, scaler = preprocess_data(stock_data, seq_length=seq_length)

    # Get the corresponding dates for the test set
    dates = stock_data.index[seq_length:]  # Dates aligned with the sequences

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Adjust the corresponding dates for the test set
    test_dates = dates[-len(X_test):]

    # Create and train the LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], 1))
    model = train_lstm_model(model, X_train, y_train, epochs=10)

    # Make predictions and evaluate the model
    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)

    # Rescale the actual test prices
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Simulate trading based on model predictions and include dates
    trade_log, profit_loss = simulate_trading(predictions.flatten(), actual_prices.flatten(), test_dates)

    print(f"Root Mean Squared Error: {rmse}")
    print(f"Final Profit/Loss: {profit_loss}")
    for log in trade_log:
        print(log)

    return stock_data, predictions, trade_log, profit_loss

# Example usage:
if __name__ == "__main__":
    ticker_symbol = 'GOOG '  # Example stock ticker (Apple Inc.)
    stock_data, predictions, trade_log, profit_loss = run_stock_prediction_with_simulation(ticker_symbol)


[*********************100%***********************]  1 of 1 completed

Epoch 1/10



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - loss: 0.0729
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0096
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 0.0046
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0037
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0030
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0027
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0030
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 0.0029
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - loss: 0.0025
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - loss: 0.0025
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
Root Mean Squared Error: 7.104833450695313
Final Profit/Loss: 1329.2990417480505
Bought 75.0 shares at 133.32000732421878 on 2023-12-01 00:00:00+00:00, Balance: 0.999450683591931, Shares: 75.0
Sold 75.0 shares at 136.63999938964847 on 2023-12-08 00:00:00+00:00, Balance: 10248.999404907227
Bought 71.0 shares at

In [ ]:
!pip install pytrends

In [ ]:
from pytrends.request import TrendReq


# Step 1.1: Download Google Trends data with additional error handling
def download_trends_data(keyword, start_date, end_date):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([keyword], timeframe=f'{start_date} {end_date}')
    trends_data = pytrends.interest_over_time()


    # Check if trends data is empty
    if trends_data.empty:
        print(f"No Google Trends data found for keyword: {keyword}")
        return None

    # Fill missing values in trends data, if any
    trends_data = trends_data.fillna(0)  # You can change the filling strategy as needed

    return trends_data


from pytrends.request import TrendReq
from textblob import TextBlob
import pandas as pd
import numpy as np
import requests

# Step 1.1: Download Google Trends data with additional error handling and sentiment analysis
def download_trends_data(keyword, start_date, end_date):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([keyword], timeframe=f'{start_date} {end_date}')
    trends_data = pytrends.interest_over_time()

    # Check if trends data is empty
    if trends_data.empty:
        print(f"No Google Trends data found for keyword: {keyword}")
        return None

    # Fill missing values in trends data, if any
    trends_data = trends_data.fillna(0)

    # Calculate sentiment based on trend score fluctuations
    trends_data['change'] = trends_data[keyword].diff()  # Daily change in interest score
    trends_data['sentiment'] = trends_data['change'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

    return trends_data

# Step 2: Preprocess the stock and trends data together
def preprocess_data_with_trends(stock_data, trends_data, feature_col='Close', seq_length=60):
    if trends_data is None:
        raise ValueError("No trends data available, cannot proceed with preprocessing.")

    trends_data.index = trends_data.index.tz_localize(None)
    stock_data.index = stock_data.index.tz_localize(None)

    # Reindex trends data to match stock data dates
    trends_data = trends_data.reindex(stock_data.index, method='ffill').fillna(0)

    # Normalize stock prices and trends data separately
    stock_scaler = MinMaxScaler(feature_range=(0, 1))
    trend_scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_stock_data = stock_scaler.fit_transform(stock_data[[feature_col]])
    scaled_trends_data = trend_scaler.fit_transform(trends_data[['sentiment']])

    # Combine the scaled stock prices and trends data
    combined_data = np.hstack((scaled_stock_data, scaled_trends_data))

    # Create sequences for LSTM input
    X, y = [], []
    for i in range(seq_length, len(combined_data)):
        X.append(combined_data[i-seq_length:i])
        y.append(scaled_stock_data[i, 0])  # Stock price (first column)

    X, y = np.array(X), np.array(y)

    return X, y, stock_scaler

# Example function to simulate trading with more realistic features
def simulate_trading_with_strategy(predictions, actual_prices, dates, initial_capital=10000, fee_rate=0.001, buy_threshold=0.01, sell_threshold=-0.01):
    capital = initial_capital
    holdings = 0  # Number of shares held
    trade_log = []
    actual_prices = actual_prices.flatten()

    for i in range(1, len(predictions)):
        predicted_change = predictions[i] - predictions[i - 1]
        actual_change = actual_prices[i] - actual_prices[i - 1]

        # Buy condition: only if the predicted increase is above the buy threshold and we have capital
        if predicted_change > buy_threshold and capital > actual_prices[i]:
            # Buy as many shares as possible within the available capital
            shares_to_buy = capital // actual_prices[i]
            capital -= shares_to_buy * actual_prices[i] * (1 + fee_rate)  # Deduct cost with fee
            holdings += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_prices[i]:.2f} on {dates[i]}")

        # Sell condition: only if the predicted decrease is below the sell threshold and we have holdings
        elif predicted_change < sell_threshold and holdings > 0:
            capital += holdings * actual_prices[i] * (1 - fee_rate)  # Sell all holdings, account for fee
            trade_log.append(f"Sold {holdings} shares at {actual_prices[i]:.2f} on {dates[i]}")
            holdings = 0  # Reset holdings after selling

    # Calculate final portfolio value
    final_value = capital + holdings * actual_prices[-1]
    profit_loss = final_value - initial_capital

    return trade_log, profit_loss, final_value

# # Example usage of the function
# trade_log, profit_loss, final_value = simulate_trading_with_strategy(predictions, actual_prices, test_dates)
# print(f"Final Profit/Loss: {profit_loss}")
# print(f"Final Portfolio Value: {final_value}")
# for log in trade_log:
#     print(log)


# Step 7: Run the entire pipeline with trading simulation, including trends data
def run_stock_prediction_with_simulation_and_trends(ticker, keyword, period='5y', seq_length=60):
    # Download stock data and Google trends data
    stock_data = download_stock_data(ticker, period)
    start_date = stock_data.index[0].strftime('%Y-%m-%d')
    end_date = stock_data.index[-1].strftime('%Y-%m-%d')

    trends_data = download_trends_data(keyword, start_date, end_date)
    if trends_data is None:
        raise ValueError("Google Trends data not available. Exiting...")

    # Preprocess the data (both stock and trends)
    X, y, scaler = preprocess_data_with_trends(stock_data, trends_data, seq_length=seq_length)

    # Get the corresponding dates for the test set
    dates = stock_data.index[seq_length:]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Adjust the corresponding dates for the test set
    test_dates = dates[-len(X_test):]

    # Create and train the LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    model = train_lstm_model(model, X_train, y_train, epochs=50)

    # Make predictions and evaluate the model
    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)
    print(f"Root Mean Squared Error: {rmse}")

    # Rescale the actual test prices
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # # Simulate trading based on model predictions and include dates
    # trade_log, profit_loss = simulate_trading(predictions.flatten(), actual_prices.flatten(), test_dates)

    # print(f"Root Mean Squared Error: {rmse}")
    # print(f"Final Profit/Loss: {profit_loss}")
    # for log in trade_log:
    #     print(log)

    trade_log, profit_loss, final_value = simulate_trading_with_strategy(predictions, actual_prices, test_dates)
    print(f"Final Profit/Loss: {profit_loss}")
    print(f"Final Portfolio Value: {final_value}")
    for log in trade_log:
        print(log)

    return stock_data, predictions, trade_log, profit_loss

# Example usage:
if __name__ == "__main__":
    ticker_symbol = 'GOOG'  # Google stock ticker for Yahoo Finance
    keyword = 'Google'  # Keyword for Google Trends to reflect broader market sentiment
    stock_data, predictions, trade_log, profit_loss = run_stock_prediction_with_simulation_and_trends(ticker_symbol, keyword)
    # 1584.8979492187518


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - loss: 0.0856
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0107
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - loss: 0.0053
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0041
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0033
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0030
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0026
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0023
Epoch 9/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0023
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0027
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0027
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0021
Epoch 13/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - loss: 0.0022
Epoch 14/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0025
Epoch 15/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0024
Ep

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sid = SentimentIntensityAnalyzer()


# Step 1.1: Fetch Google Trends data as a sentiment proxy
def fetch_trends_data_for_keyword(keyword, start_date, end_date):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([keyword], timeframe=f'{start_date} {end_date}')
    trends_data = pytrends.interest_over_time()

    # Check if trends data is empty
    if trends_data.empty:
        print(f"No Google Trends data found for keyword: {keyword}")
        return None

    # Process trends data as a sentiment proxy
    trends_data = trends_data[keyword]  # Select the interest data for the keyword
    trends_data = trends_data.fillna(0)  # Fill missing values, if any

    return trends_data

# Step 1.2: Updated function to calculate sentiment scores based on trends data (using trend level as sentiment proxy)
def download_sentiment_data(keyword, start_date, end_date):
    trends_data = fetch_trends_data_for_keyword(keyword, start_date, end_date)
    if trends_data is None:
        return None

    # Calculate daily average as sentiment proxy (optional adjustment)
    sentiment_series = trends_data.resample('D').mean().fillna(0)  # Fill missing dates with zero interest
    return sentiment_series


# Step 2: Preprocess stock and trends data together, adding sentiment data
def preprocess_data_with_trends_and_sentiment(stock_data, trends_data, sentiment_data, feature_col='Close', seq_length=60):
    if trends_data is None or sentiment_data is None:
        raise ValueError("Trends or sentiment data not available, cannot proceed with preprocessing.")

    trends_data.index = trends_data.index.tz_localize(None)
    stock_data.index = stock_data.index.tz_localize(None)
    sentiment_data.index = sentiment_data.index.tz_localize(None)

    # Reindex trends and sentiment data to match stock data dates
    trends_data = trends_data.reindex(stock_data.index, method='ffill').fillna(0)
    sentiment_data = sentiment_data.reindex(stock_data.index, method='ffill').fillna(0)

    # Normalize stock prices, trends, and sentiment data separately
    stock_scaler = MinMaxScaler(feature_range=(0, 1))
    trend_scaler = MinMaxScaler(feature_range=(0, 1))
    sentiment_scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_stock_data = stock_scaler.fit_transform(stock_data[[feature_col]])
    scaled_trends_data = trend_scaler.fit_transform(trends_data[[keyword]])
    scaled_sentiment_data = sentiment_scaler.fit_transform(sentiment_data.values.reshape(-1, 1))

    # Combine scaled stock prices, trends, and sentiment data
    combined_data = np.hstack((scaled_stock_data, scaled_trends_data, scaled_sentiment_data))

    # Create sequences for LSTM input
    X, y = [], []
    for i in range(seq_length, len(combined_data)):
        X.append(combined_data[i-seq_length:i])
        y.append(scaled_stock_data[i, 0])  # Stock price (first column)

    X, y = np.array(X), np.array(y)

    return X, y, stock_scaler

# Example usage in the pipeline
def run_stock_prediction_with_sentiment(ticker, keyword, period='5y', seq_length=60):
    stock_data = download_stock_data(ticker, period)
    start_date = stock_data.index[0].strftime('%Y-%m-%d')
    end_date = stock_data.index[-1].strftime('%Y-%m-%d')

    trends_data = download_trends_data(keyword, start_date, end_date)
    sentiment_data = download_sentiment_data(keyword, start_date, end_date)

    X, y, scaler = preprocess_data_with_trends_and_sentiment(stock_data, trends_data, sentiment_data, seq_length=seq_length)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    test_dates = stock_data.index[seq_length + len(X_train):]


    model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    model = train_lstm_model(model, X_train, y_train, epochs=50)

    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)
    print(f"Root Mean Squared Error: {rmse}")

        # Rescale the actual test prices for trading simulation
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Run the trading simulation based on predictions
    trade_log, profit_loss, final_value = simulate_trading_with_strategy(predictions.flatten(), actual_prices, test_dates)

    # Output results
    print(f"Final Profit/Loss: {profit_loss}")
    print(f"Final Portfolio Value: {final_value}")
    for log in trade_log:
        print(log)

    return stock_data, predictions, trade_log, profit_loss


# Run example
if __name__ == "__main__":
    ticker_symbol = 'GOOG'
    keyword = 'Google'
    run_stock_prediction_with_sentiment(ticker_symbol, keyword)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass 

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 111ms/step - loss: 0.1156
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - loss: 0.0103
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0045
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0034
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0032
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0027
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0029
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0025
Epoch 9/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0025
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0024
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 0.0022
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - loss: 0.0020
Epoch 13/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0022
Epoch 14/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0021
Epoch 15/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0020
